In [421]:
import $ivy.`org.apache.spark::spark-sql:2.4.3` // Or use any other 2.x version here
import $ivy.`sh.almond::almond-spark:_` // Added automatically on importing Spark
import org.apache.spark.sql._
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.sql.Column
import org.apache.spark.sql.types.{DataType, DateType, TimestampType}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.DataFrameNaFunctions
import org.apache.spark.sql.DataFrameStatFunctions


var spark = SparkSession
.builder()
.appName("Spark HW 1")
.config("spark.master", "local")
.getOrCreate();

def sc = spark.sparkContext

val df = spark.read.
option("header", true).
option("inferSchema",true).
csv("./PicsArtData.csv").
withColumn("date", to_date(col("timestamp")))

val dfSampled = spark.read.
option("header", true).
option("inferSchema",true).
csv("./SampledData.csv").
withColumn("date", to_date(col("timestamp")))

import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                           // Added automatically on importing Spark

import org.apache.spark.sql._

import org.apache.log4j.{Level, Logger}

import org.apache.spark.SparkContext

import org.apache.spark.SparkConf

import org.apache.spark.sql.Column

import org.apache.spark.sql.types.{DataType, DateType, TimestampType}

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.DataFrameNaFunctions

import org.apache.spark.sql.DataFrameStatFunctions



spark: SparkSession = org.apache.spark.sql.SparkSession@553478b8
defined function sc
df: DataFrame = [country_code: string, device_id: string ... 13 more fields]
dfSampled: DataFrame = [device_id: string, country_code: string ... 13 more fields]

In [390]:
/*  
    1.  Calculate how many users who used app in X day 
    a)  returned the next day
    b)  returned 7 day after
*/

val window_func = Window.partitionBy("device_id").orderBy("date")

dfSampled.select("device_id", "date").distinct().
withColumn("next_day", lead("date", 1) over window_func).
withColumn("7_days", lead("date", 7) over window_func).
groupBy("date").
agg(
    count("device_id").alias("daily_users"),
    sum(when(col("next_day").isin(date_add(col("next_day"), 1)), 1)).alias("next_day_return"),
    sum(when(col("7_days").isin(date_add(col("7_days"), 7)), 1)).alias("7_day_return")).
orderBy("date").show(100, false)

/*
    The when clause's conditional statement was not working and that is why the columns have null.
    I am not sure why it's not working though, maybe a type error?
*/

+----------+-----------+---------------+------------+
|date      |daily_users|next_day_return|7_day_return|
+----------+-----------+---------------+------------+
|2018-02-02|94         |null           |null        |
|2018-02-03|41         |null           |null        |
|2018-02-04|39         |null           |null        |
|2018-02-05|45         |null           |null        |
|2018-02-06|35         |null           |null        |
|2018-02-07|34         |null           |null        |
|2018-02-08|36         |null           |null        |
|2018-02-09|29         |null           |null        |
|2018-02-10|31         |null           |null        |
|2018-02-11|28         |null           |null        |
|2018-02-12|33         |null           |null        |
|2018-02-13|29         |null           |null        |
|2018-02-14|40         |null           |null        |
|2018-02-15|33         |null           |null        |
|2018-02-16|30         |null           |null        |
|2018-02-17|20         |null

window_func: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@3ad9dfe4

In [402]:
/*  
    2.  Which tool has highest return rate (edit_text_apply, effect_apply,
        edit_item_open, sticker_apply)
*/

val window_func = Window.partitionBy("device_id").orderBy("date")

val edTxtApply = df.select("device_id", "date", "event_type").distinct().
filter(col("event_type") === "edit_text_apply").
withColumn("converted", lead("date", 1) over window_func).
groupBy("date").agg(count("converted").alias("ed_txt_apply return")).
na.drop().agg(avg("ed_txt_apply return")).show

val efApply = df.select("device_id", "date", "event_type").distinct().
filter(col("event_type") === "effect_apply").
withColumn("converted", lead("date", 1) over window_func).
groupBy("date").agg(count("converted").alias("effect_apply return")).
na.drop().agg(avg("effect_apply return")).show

val edItmOpen = df.select("device_id", "date", "event_type").distinct().
filter(col("event_type") === "edit_item_open").
withColumn("converted", lead("date", 1) over window_func).
groupBy("date").agg(count("converted").alias("ed_itm_open return")).
na.drop().agg(avg("ed_itm_open return")).show

val stckrApply = df.select("device_id", "date", "event_type").distinct().
filter(col("event_type") === "sticker_tap_apply").
withColumn("converted", lead("date", 1) over window_func).
groupBy("date").agg(count("converted").alias("stckr_apply return")).
na.drop().agg(avg("stckr_apply return")).show

/* 
    edit_item_open has the highest return on average and edit_text_apply, 
    sticker_tap_apply both have the lowest returns
*/

+------------------------+
|avg(ed_txt_apply return)|
+------------------------+
|                     0.0|
+------------------------+

+------------------------+
|avg(effect_apply return)|
+------------------------+
|     0.03333333333333333|
+------------------------+

+-----------------------+
|avg(ed_itm_open return)|
+-----------------------+
|     0.6333333333333333|
+-----------------------+

+-----------------------+
|avg(stckr_apply return)|
+-----------------------+
|                    0.0|
+-----------------------+



window_func: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@a38723a

In [393]:
/*
    3.  How many users did photo_like daily, in which day is it highest, if there is seasonality
*/

// 1 = Sunday, 2 = Monday, 3 = Tuesday, 4 = ...
df.
filter(col("event_type") === "photo_like").
groupBy("date").
agg(countDistinct("device_id").as("daily_PhotoLike_users")).
withColumn("weekday", dayofweek(col("date")).as("weekday")).
orderBy("date").
show(100, false)

df.
filter(col("event_type") === "photo_like").
groupBy(dayofweek(col("date")).as("weekday")).
agg(countDistinct("device_id").as("PhotoLike_users")).
orderBy("weekday").
show(100, false)

/*  
    Surprisingly, Tuesday has the most usage of "photo likes", followed by Friday and Saturday.
    I would have expected Fri/Sat to have the most users who "liked" photos, since those days have the
    most active users overall. Tuesday is kind of a random weekday
*/ 

+----------+---------------------+-------+
|date      |daily_PhotoLike_users|weekday|
+----------+---------------------+-------+
|2018-02-02|6                    |6      |
|2018-02-03|3                    |7      |
|2018-02-04|5                    |1      |
|2018-02-05|1                    |2      |
|2018-02-06|5                    |3      |
|2018-02-07|4                    |4      |
|2018-02-08|3                    |5      |
|2018-02-09|1                    |6      |
|2018-02-10|4                    |7      |
|2018-02-11|1                    |1      |
|2018-02-12|1                    |2      |
|2018-02-13|7                    |3      |
|2018-02-14|4                    |4      |
|2018-02-15|2                    |5      |
|2018-02-16|5                    |6      |
|2018-02-17|4                    |7      |
|2018-02-18|5                    |1      |
|2018-02-19|1                    |2      |
|2018-02-20|7                    |3      |
|2018-02-21|4                    |4      |
|2018-02-22

In [407]:
/*
    4.  What is rate that users who did effect_try then did effect apply
*/

val pivoted = dfSampled.
filter(col("event_type") === "effect_try" || col("event_type") === "effect_apply").
groupBy("session_id").
pivot("event_type").
agg(count("device_id")).
withColumn("Conversion (%)", round(col("effect_apply")/col("effect_try")*100, 2)).
orderBy(asc("Conversion (%)")).na.drop()

pivoted.show(50, false)
pivoted.agg(round(avg(col("Conversion (%)")), 2).alias("Avg. Conversion Rate (%)")).show()

+-------------------------------------------------------+------------+----------+--------------+
|session_id                                             |effect_apply|effect_try|Conversion (%)|
+-------------------------------------------------------+------------+----------+--------------+
|1517608781522_andy-4e62fd2d-b60b-4f3c-973a-bce9cc1998d7|1           |76        |1.32          |
|1517587340835_1cb08ce0-435b-4458-977e-a11572a8ff04     |1           |50        |2.0           |
|1518676219300_76fe568f-5fd7-4354-bf73-6c4a1847be83     |1           |43        |2.33          |
|1517803973643_andy-089c1aad-1f55-481e-9934-aac43613239d|1           |39        |2.56          |
|1518238636684_andy-cf9696c3-d472-4dc8-b7bd-ecabe23ea56d|1           |31        |3.23          |
|1518103739373_7ad2ab57-2815-4434-9ae4-d000f8e93805     |1           |30        |3.33          |
|1518179217782_andy-089c1aad-1f55-481e-9934-aac43613239d|1           |27        |3.7           |
|1517643629326_aafbc2e7-5bd3-4

pivoted: DataFrame = [session_id: string, effect_apply: bigint ... 2 more fields]

In [427]:
/*
    5.  Find how many users did effect apply and what is descriptive
        statistics for it (outliers, avg, median)
*/

df.
filter(col("event_type") === "effect_apply").
groupBy("date").
agg(countDistinct(col("device_id")).as("daily_users")).
select(sum("daily_users").as("monthly users of effect_apply"),
      max("daily_users").as("max users in a day"), min("daily_users").as("min users in a day"),
      avg("daily_users").as("avg users per day")).
show(100, false)

val median = df.
filter(col("event_type") === "effect_apply").
groupBy("date").
agg(countDistinct(col("device_id")).as("daily_users")).stat.approxQuantile("daily_users", Array(0.5), 0)


+-----------------------------+------------------+------------------+------------------+
|monthly users of effect_apply|max users in a day|min users in a day|avg users per day |
+-----------------------------+------------------+------------------+------------------+
|4081                         |181               |16                |136.03333333333333|
+-----------------------------+------------------+------------------+------------------+



median: Array[Double] = Array(138.0)

In [420]:
/*
    6.  What first 5 events users do in their session start (should be
        used window functions, find about it in the shared link above) - 5
        events in array and % of users did it
*/


var window_func = Window.partitionBy("device_id").orderBy("date")
val totalUsers = countDistinct("device_id")

dfSampled.
withColumn("ev1", lead("event_type", 1) over window_func).
withColumn("ev2", lead("event_type", 2) over window_func).
withColumn("ev3", lead("event_type", 3) over window_func).
withColumn("ev4", lead("event_type", 4) over window_func).
withColumn("ev5", lead("event_type", 5) over window_func).
filter(col("event_type") === "app_open").
withColumn("first_5_events_after_app-open",concat(col("event_type"),lit(","),
                                                  col("ev1"),lit(","),col("ev2"),lit(","),
                                                  col("ev3"),lit(","),col("ev4"),lit(","),
                                                  col("ev5"))).
select("device_id","first_5_events_after_app-open").
groupBy("first_5_events_after_app-open").
agg(count("device_id").as("users")).
// withColumn("perc_of_users", (col("users")/totalUsers)).
orderBy(col("users").desc).na.drop().
show(30,false)


+---------------------------------------------------------------------------------------------------------------------------+-----+
|first_5_events_after_app-open                                                                                              |users|
+---------------------------------------------------------------------------------------------------------------------------+-----+
|app_open,explore_open,interstitial_ad_request,interstitial_ad_request,app_load,native_ad_request                           |104  |
|app_open,app_open,app_open,app_open,app_open,app_open                                                                      |52   |
|app_open,explore_open,app_load,interstitial_ad_request,banner_ad_request,interstitial_ad_request                           |47   |
|app_open,explore_open,interstitial_ad_request,app_load,native_ad_request,native_ad_request                                 |22   |
|app_open,explore_open,app_load,interstitial_ad_request,native_ad_request,in

window_func: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@2c21bd2b
totalUsers: Column = count(DISTINCT device_id)

In [ ]:
/*
    7.  What are the most important metrics when we want to describe 
        one session of user in app
*/

//            I think the most important metrics would be:
//     1.) How long the app was used by the user in that one session
//     2.) The chronologial order of events completed by the user during that one session 
